In [14]:
import os
import json
import csv
import pandas as pd
from collections import defaultdict, Counter

# JSON 파일이 있는 디렉토리 경로
train_directory = '../dataset/training_label'
valid_directory = '../dataset/validation_label'
train_image_directory = '../dataset/training_image'
valid_image_directory = '../dataset/validation_image'

# 결과를 저장할 딕셔너리
train_result = defaultdict(lambda: {'선호': [], '비선호': []})
valid_result = defaultdict(lambda: {'선호': [], '비선호': []})
response_count = Counter()

# 유효한 이미지 ID를 얻기 위한 함수
def count_images_by_gender_and_style(file_names, valid_image_ids):
    rows = []  # 결과를 저장할 리스트

    # 파일명 분석 및 카운트
    for filename in file_names:
        # 파일 확장자 체크
        if not filename.endswith('.json'):
            continue

        # .json 제거
        no_jpg_filename = filename.split('.')[0]
        parts = no_jpg_filename.split('_')

        # 성별, 스타일, 이미지 ID 추출
        gender = '여성' if parts[4] == 'W' else '남성'
        style = parts[3]  
        image_id = parts[1] 

        # 유효한 스타일과 이미지 ID일 때만 추가
        if style is not None and image_id is not None and image_id in valid_image_ids:
            rows.append({'성별': gender, '스타일': style, '이미지 ID': image_id})

    # DataFrame 생성
    df = pd.DataFrame(rows)

    # 중복된 행 제거
    df = df.drop_duplicates(subset=['성별', '스타일', '이미지 ID'])

    # 성별과 스타일별로 이미지 수 집계
    result = df.groupby(['성별', '스타일']).size().reset_index(name='이미지 수')
    result = result.sort_values(by='성별')
    
    return result

# 유효한 이미지 ID를 얻기 위한 파일 목록
train_files = os.listdir(train_directory)
valid_files = os.listdir(valid_directory)
train_image_files = os.listdir(train_image_directory)
valid_image_files = os.listdir(valid_image_directory)

# 이미지 파일에서 유효한 이미지 ID 추출
valid_image_ids = set([filename.split('_')[1] for filename in train_image_files + valid_image_files if filename.endswith('.jpg')])

# 유효한 데이터만 필터링
train_valid_images = count_images_by_gender_and_style(train_files + valid_files, valid_image_ids)

# 디렉토리 내의 모든 파일을 순회하며 데이터 추출
def process_directory(directory, result_dict, valid_image_ids):
    for filename in os.listdir(directory):
        if filename.endswith('.json'):
            filepath = os.path.join(directory, filename)
            with open(filepath, 'r', encoding='utf-8') as file:
                data = json.load(file)
                R_id = data['user']['R_id']
                imgName = data['item']['imgName']
                Q5 = data['item']['survey']['Q5']
                
                # 이미지 ID 추출
                image_id = imgName.split('_')[1]
                
                # 유효한 이미지 ID일 때만 처리
                if image_id in valid_image_ids:
                    # Q5 값을 선호도 값으로 변환
                    preference = "선호" if Q5 == 2 else "비선호"
                    
                    result_dict[R_id][preference].append(imgName)
                    response_count[R_id] += 1

# train과 valid 디렉토리 처리
process_directory(train_directory, train_result, valid_image_ids)
process_directory(valid_directory, valid_result, valid_image_ids)

# 결과를 CSV 파일로 저장
with open('mission2_2_result.csv', 'w', newline='', encoding='utf-8-sig') as csvfile:
    fieldnames = ['응답자 ID', 'train 선호', 'train 비선호', 'valid 선호', 'valid 비선호']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
    writer.writeheader()
    for R_id in response_count:
        train_preference = ', '.join(train_result[R_id]['선호'])
        train_non_preference = ', '.join(train_result[R_id]['비선호'])
        valid_preference = ', '.join(valid_result[R_id]['선호'])
        valid_non_preference = ', '.join(valid_result[R_id]['비선호'])
        
        writer.writerow({
            '응답자 ID': R_id,
            'train 선호': train_preference,
            'train 비선호': train_non_preference,
            'valid 선호': valid_preference,
            'valid 비선호': valid_non_preference
        })

In [15]:
import pandas as pd

# 출력 옵션 설정
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_columns', None)

# CSV 파일 읽기
result = pd.read_csv("mission2_2_result.csv")

In [16]:
len(result)

4116

In [17]:
# 상위 100개 행 선택
top_100 = result.head(100)

# CSV 파일로 저장
top_100.to_csv('top_100_mission2_2_result.csv', index=False)

In [18]:
import pandas as pd

# CSV 파일 읽기
top_100_result = pd.read_csv("top_100_mission2_2_result.csv")

In [19]:
len(top_100_result)

100

In [20]:
top_100_result.head(3)


,응답자 ID,train 선호,train 비선호,valid 선호,valid 비선호
0,52002,W_24111_70_hippie_M.jpg,"T_00004_90_hiphop_M.jpg, T_03007_10_sportivecasual_M.jpg, T_03118_19_normcore_M.jpg, W_02699_60_mods_M.jpg, W_23983_60_mods_M.jpg",NaN,NaN
1,66699,"T_00004_90_hiphop_M.jpg, T_01568_50_ivy_M.jpg, T_15877_10_sportivecasual_M.jpg, T_16259_10_sportivecasual_M.jpg, W_00901_60_mods_M.jpg","T_03643_00_metrosexual_M.jpg, T_06009_10_sportivecasual_M.jpg, W_15364_00_metrosexual_M.jpg, W_24031_60_mods_M.jpg",NaN,NaN
2,66797,"T_01259_10_sportivecasual_M.jpg, T_16092_10_sportivecasual_M.jpg, T_16332_10_sportivecasual_M.jpg, W_23958_60_mods_M.jpg, W_24020_60_mods_M.jpg","T_00004_90_hiphop_M.jpg, W_15467_70_hippie_M.jpg, W_24865_60_mods_M.jpg, W_49510_00_metrosexual_M.jpg","T_08486_10_sportivecasual_M.jpg, W_23958_60_mods_M.jpg",NaN


.